# Skill 2 : Data Analysis from CSV file

To really have a Smart Search Engine or Copilot/Virtual assistant that can answer any question about your corporate documents, this "engine" must understand tabular data, aka, sources with tables, rows and columns with numbers. 

This is a totally different problem that simply looking for the top most semantically similar results.  The concept of indexing, bringing top results, embedding, doing a cosine semantic search and summarize an answer, doesn't really apply to this problem.
We are dealing now with sources with Tables in which each row and column are related to each other, and in order to answer a question, all of the data is needed, not just top results.

In this notebook, the goal is to show how to deal with this kind of use cases. To continue with our Covid-19 theme, we will be using an open dataset called ["Covid Tracking Project"](https://covidtracking.com/data/download). The COVID Tracking Project dataset is a  CSV file that provides the latest numbers on tests, confirmed cases, hospitalizations, and patient outcomes from every US state and territory (they stopped tracking on March 7 2021).

Imagine that many documents on a data lake are tabular data, or that your use case is to ask questions in natural language to a LLM model and this model needs to get the context from a CSV file or even a SQL Database in order to answer the question. A GPT Smart Search Engine, must understand how to deal with this sources, understand the data and answer acoordingly.

In [1]:
import os
import pandas as pd
from langchain_openai import AzureChatOpenAI

from langchain_experimental.tools import PythonREPLTool, PythonAstREPLTool
from langgraph.prebuilt import create_react_agent

from common.prompts import CSV_AGENT_PROMPT_TEXT

from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials.env")

def printmd(string):
    display(Markdown(string))

In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

## Download the dataset and load it into Pandas Dataframe

In [3]:
os.makedirs("data",exist_ok=True)

In [4]:
!wget https://covidtracking.com/data/download/all-states-history.csv -P ./data/

--2025-01-02 03:34:56--  https://covidtracking.com/data/download/all-states-history.csv
Resolving covidtracking.com (covidtracking.com)... 104.21.64.114, 172.67.183.132, 2606:4700:3032::ac43:b784, ...
Connecting to covidtracking.com (covidtracking.com)|104.21.64.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘./data/all-states-history.csv.1’

all-states-history.     [ <=>                ]   2.61M  --.-KB/s    in 0.06s   

2025-01-02 03:34:57 (46.3 MB/s) - ‘./data/all-states-history.csv.1’ saved [2738601]



In [5]:
file_url = "./data/all-states-history.csv"
df = pd.read_csv(file_url).fillna(value = 0)
print("Rows and Columns:",df.shape)
df.head()

Rows and Columns: (20780, 41)


,date,state,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
0,2021-03-07,AK,305.0,0.0,0,0.0,1293.0,1293.0,33.0,0,...,1731628.0,0,0.0,0.0,0.0,0.0,0.0,0,1731628.0,0
1,2021-03-07,AL,10148.0,7963.0,-1,2185.0,45976.0,45976.0,494.0,0,...,2323788.0,2347,0.0,0.0,119757.0,0.0,2323788.0,2347,0.0,0
2,2021-03-07,AR,5319.0,4308.0,22,1011.0,14926.0,14926.0,335.0,11,...,2736442.0,3380,0.0,0.0,0.0,481311.0,0.0,0,2736442.0,3380
3,2021-03-07,AS,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,2140.0,0,0.0,0.0,0.0,0.0,0.0,0,2140.0,0
4,2021-03-07,AZ,16328.0,14403.0,5,1925.0,57907.0,57907.0,963.0,44,...,7908105.0,45110,580569.0,0.0,444089.0,0.0,3842945.0,14856,7908105.0,45110


In [6]:
df.columns

Index(['date', 'state', 'death', 'deathConfirmed', 'deathIncrease',
       'deathProbable', 'hospitalized', 'hospitalizedCumulative',
       'hospitalizedCurrently', 'hospitalizedIncrease', 'inIcuCumulative',
       'inIcuCurrently', 'negative', 'negativeIncrease',
       'negativeTestsAntibody', 'negativeTestsPeopleAntibody',
       'negativeTestsViral', 'onVentilatorCumulative', 'onVentilatorCurrently',
       'positive', 'positiveCasesViral', 'positiveIncrease', 'positiveScore',
       'positiveTestsAntibody', 'positiveTestsAntigen',
       'positiveTestsPeopleAntibody', 'positiveTestsPeopleAntigen',
       'positiveTestsViral', 'recovered', 'totalTestEncountersViral',
       'totalTestEncountersViralIncrease', 'totalTestResults',
       'totalTestResultsIncrease', 'totalTestsAntibody', 'totalTestsAntigen',
       'totalTestsPeopleAntibody', 'totalTestsPeopleAntigen',
       'totalTestsPeopleViral', 'totalTestsPeopleViralIncrease',
       'totalTestsViral', 'totalTestsViralIncrease'

## CSV/Pandas Agent

LLMs are great for building question-answering systems over various types of data sources. In this section we’ll go over how to build Q&A systems over data stored in a CSV file(s). Like we did in the last notebook, the key to working with tabular CSV files is to give an LLM access to tools/experts for querying and interacting with the data

In [7]:
# Let's delve into a challenging question that demands a multi-step solution. 
# The path to solving it might not be immediately clear.
# When examining the dataframe above, even a human might struggle to determine which columns are pertinent.

QUESTION = f"""
How may patients were hospitalized during July 2020 in Texas. 
And nationwide as the total of all states? 
"""

In [8]:
# First we load our LLM
COMPLETION_TOKENS = 1000
llm = AzureChatOpenAI(deployment_name=os.environ["GPT4o_DEPLOYMENT_NAME"], 
                      temperature=0.5, max_tokens=COMPLETION_TOKENS)

### Define our expert tool

In [9]:
# This executes code locally, which can be unsafe
tools = [PythonAstREPLTool()]

### Define our instructions for the Agent

In [10]:
printmd(CSV_AGENT_PROMPT_TEXT)



## Source of Information
- Use the data in this CSV filepath: {file_url}

## On how to use the Tool
- You are an agent designed to write and execute python code to answer questions from a CSV file.
- Given the path to the csv file, start by importing pandas and creating a df from the csv file.
- First set the pandas display options to show all the columns, get the column names, see the first (head(5)) and last rows (tail(5)), describe the dataframe, so you have an understanding of the data and what column means. Then do work to try to answer the question.
- **ALWAYS** before giving the Final Answer, try another method. Then reflect on the answers of the two methods you did and ask yourself if it answers correctly the original question. If you are not sure, try another method.
- If the methods tried do not give the same result, reflect and try again until you have two methods that have the same result. 
- If you still cannot arrive to a consistent result, say that you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE Pre-Existing KNOWLEDGE, ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**. 
- If you get an error, debug your code and try again, do not give python code to the  user as an answer.
- Only use the output of your code to answer the question. 
- You might know the answer without running any code, but you should still run the code to get the answer.
- If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.
- **ALWAYS**, as part of your "Final Answer", explain thoroughly how you got to the answer on a section that starts with: "Explanation:". In the explanation, mention the column names that you used to get to the final answer. 


### Create the Graph
    
There is a prebuild graph called `create_react_agent` [HERE](https://langchain-ai.github.io/langgraph/reference/prebuilt/), that creates a graph that works with a chat model that utilizes tool calling.

Simple agents can use this prebuilt graph without having to write all the code we did on the prior Notebook.

LangGraph's prebuilt **create_react_agent** does not take a prompt template directly as a parameter, but instead takes a `state_modifier` parameter. This modifies the graph state before the llm is called, and can be one of four values:

- SystemMessage, which is added to the beginning of the list of messages.
- string, which is converted to a SystemMessage and added to the beginning of the list of messages.
- Callable, which should take in full graph state. The output is then passed to the language model.
- Runnable, which should take in full graph state. The output is then passed to the language model.



In [11]:
graph = create_react_agent(llm, tools=tools, state_modifier=CSV_AGENT_PROMPT_TEXT.format(file_url=file_url))

### Run the Graph

In [12]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [13]:
inputs = {"messages": [("user", QUESTION)]}

print_stream(graph.stream(inputs, stream_mode="values"))

================================ Human Message =================================


How may patients were hospitalized during July 2020 in Texas. 
And nationwide as the total of all states? 

================================== Ai Message ==================================
Tool Calls:
  python_repl_ast (call_Xp7450tdBoLvJNsPGps2gCNA)
 Call ID: call_Xp7450tdBoLvJNsPGps2gCNA
  Args:
    query: import pandas as pd

df = pd.read_csv('./data/all-states-history.csv')
pd.set_option('display.max_columns', None)
df.head()
================================= Tool Message =================================
Name: python_repl_ast

         date state    death  deathConfirmed  deathIncrease  deathProbable  \
0  2021-03-07    AK    305.0             NaN              0            NaN   
1  2021-03-07    AL  10148.0          7963.0             -1         2185.0   
2  2021-03-07    AR   5319.0          4308.0             22         1011.0   
3  2021-03-07    AS      0.0             NaN              0        

That was good pandas code, and pretty fast!.

## Evaluation
Let's see if the answer is correct

In [14]:
#df['date'] = pd.to_datetime(df['date'])
july_2020 = df[(df['date'] >= '2020-07-01') & (df['date'] <= '2020-07-31')]
texas_hospitalized_july_2020 = july_2020[july_2020['state'] == 'TX']['hospitalizedIncrease'].sum()
nationwide_hospitalized_july_2020 = july_2020['hospitalizedIncrease'].sum()

In [15]:
print( "TX:",texas_hospitalized_july_2020,"Nationwide:",nationwide_hospitalized_july_2020)

TX: 0 Nationwide: 63105


It is Correct!

**Note**: Obviously, there were hospitalizations in Texas in July 2020 (Try asking ChatGPT), but this particular File, for some reason has 0 on the column "hospitalizedIncrease" for Texas in July 2020. This proves though that the model is NOT making up information or using prior knowledge, but instead using only the results of its calculation on this CSV file. That's what we need!

# Summary

We’ve successfully solved the problem of asking natural language questions to tabular data stored in a CSV file. With this approach, it's important to note that it's not necessary to dump database data into a CSV file and index it in a search engine, that won't work. 


**Important Note**: We don't recommend running an agent on the same compute environment as the main application to query tabular data. Exposing a Python REPL to users can introduce security risks. The most secure way to run code for users is through **[Azure Container Apps dynamic sessions](https://learn.microsoft.com/en-us/azure/container-apps/sessions?tabs=azure-cli)**. This service provides fast access to secure, sandboxed environments that are ideal for running code or applications with strong isolation from other workloads.

[This example](https://github.com/langchain-ai/langchain/blob/master/cookbook/azure_container_apps_dynamic_sessions_data_analyst.ipynb), created by Microsoft and LangChain, demonstrates a secure solution for this use case.

# NEXT
We can see that GPT-4o, is powerful and can translate a natural language question into the right steps in python in order to query a CSV data loaded into a pandas dataframe. 
That's pretty amazing. However the question remains: **Do I need then to dump all the data from my original sources (Databases, ERP Systems, CRM Systems) in order to be searchable by a Smart Search Engine?**

The next Notebook answers this question by implementing a Question->SQL process and get the information from data in a SQL Database, eliminating the need to dump it out.